# 实验七 空气中声速的测量

Copyright © 2023 longlin

Copyright © 2024 Zjl37

本 notebook 修改自 [HITSZ-OpenAuto/PHYS1002](https://github.com/HITSZ-OpenAuto/PHYS1002/blob/3554c6e1852fe3a9cc18beb9fd8cbcd1df761ea6/Exp07/%E5%A3%B0%E9%80%9F%E6%B5%8B%E9%87%8F%EF%BC%88by%20longlin%EF%BC%89.ipynb)，以与 HITSZ-OpenAuto 项目相同的协议授权。


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import yaml
import re

In [ ]:
# 如果你未做“波形移动法测空气中声速”部分，将下面常量改为 False
ENABLE_ST3 = True
# 如果你未做“时差法测固体中声速”部分，将下面常量改为 False
ENABLE_ST5 = True
# st is short for subtask

In [ ]:
# 从文件中读取数据
data_file = './data/data.yaml'
data = yaml.safe_load(open(data_file))
def data_to_list(ln, cast=float):
    return list(map(cast, re.split(r'[,，;；]?\s+', ln)))

In [ ]:
if not data_to_list(data['st4']['t_i'])[7]:
    raise ValueError(f'请先在 {data_file} 文件中填写好你的实验数据。')

In [ ]:
def v0_ref(t):
    """计算温度 t ℃ 时空气中声速的参考值"""
    return 331.45*math.sqrt(1+t/273.15) # (m/s)


In [ ]:
def calc(l: list[float], frequency: float, factor: float = 2):
    """使用逐差法计算 l_i 的平均差距，并输出结果波长、声速的计算结果。"""
    """l 的单位为 mm，frequency 的单位为 kHz"""
    """factor 为 λ/Δl，在“波形移动法”部分中为 1，其余部分中应为 2."""
    
    l = np.array(l)
    n = len(l)//2

    dl = (l[-n:] - l[:n]).sum()/n/(len(l)-n) # (mm)
    dl = abs(dl) # (mm)
    
    print(f"　　间隔 Δl = {round(abs(dl), 3)} mm")
    wavelen = factor*dl # (mm)
    print(f"　　波长  λ = {round(abs(wavelen), 3)} mm")
    v = wavelen*frequency # (m/s)
    print(f"　　声速  v = {round(abs(v), 3)} m/s")
    
    E = (v - v0)/v0
    print(f"相对误差  E = {E}")


## 1. 极值法（驻波法）测空气中声速

In [ ]:
for st in [data['st1']]:
    l = data_to_list(st['l_i'])
    f = float(st['f'])
    t = float(st['t'])
    v0 = v0_ref(t)
    print(f"参考声速 v0 = {round(abs(v0), 3)} m/s")
    calc(l, f)

## 2. 相位比较法测空气中声速

In [ ]:
for st in [data['st2']]:
    l = data_to_list(st['l_i'])
    f = float(st['f'])
    t = float(st['t'])
    v0 = v0_ref(t)
    print(f"参考声速 v0 = {round(abs(v0), 3)} m/s")
    calc(l, f)
    # 以上计算是按每个数据点对应的李萨如图形呈直线的斜率是正负交替的情形进行的；
    # 如果按你的实验方法，记录的每个数据点对应的李萨如图形呈直线皆沿同一方向，则用：
    # calc(l, f, factor=1)

## 3. 波形移动法测空气中声速

In [ ]:
for st in [data['st3']]*ENABLE_ST3:
    l = data_to_list(st['l_i'])
    f = float(st['f'])
    t = float(st['t'])
    v0 = v0_ref(t)
    print(f"参考声速 v0 = {round(abs(v0), 3)} m/s")
    calc(l, f, factor=1)

## 4. 时差法测空气中声速

In [ ]:
def calc_st4(l: list[float], t: list[float]):
    """“时差法测空气中声速”部分；用最小二乘法计算声速"""
    """l 的单位为 mm，t 的单位为 µs"""
    import scipy
    
    l = 1e-3*np.array(l) # (m)
    t = 1e-6*np.array(t) # (s)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(t, l) # slope: (mV/mA) 即 (V/A)

    v = slope # (m/s)
    print(f"　　声速 v = {round(abs(v), 3)} m/s")
    print(f"相关系数 r = {r_value}")
    return v
    

In [ ]:
for st in [data['st4']]:
    l = data_to_list(st['l_i'])
    t = data_to_list(st['t_i'])
    temp = float(st['t'])
    v0 = v0_ref(temp)
    print(f"参考声速 v0 = {round(abs(v0), 3)} m/s")
    v = calc_st4(l, t)
    E = (abs(v) - v0)/v0
    print(f"相对误差 E = {E}")

## 5. 时差法测固体中声速

In [ ]:
st = data['st5']
material = data_to_list(st['材质'], cast=str)
l = data_to_list(st['l_i'])
t = data_to_list(st['t_i'])

In [ ]:
for st in [data['st5']]*ENABLE_ST5:
    material = data_to_list(st['材质'], cast=str)
    l = data_to_list(st['l_i'])
    t = data_to_list(st['t_i'])
    
    material = np.array(material)
    l = np.array(l)
    t = np.array(t)
    
    for mat in set(material):
        li = l[material == mat]
        ti = t[material == mat]
        print(f"===== 材质：{mat} =====")
        calc_st4(li, ti)